# Garbage Classification using CNN Architecture
- Classify garbage into organic and reclyclable types with a aim to assis people in managing household waste more smartly.

In [1]:
#Import necesssary libraries
import os
import random
import torch
from torch import nn
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
import torch.optim as optim
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm

In [2]:
#paths for train and validation image directories
TRAIN_DIR = "/kaggle/input/waste-classification-data/DATASET/TRAIN/"
TEST_DIR = "/kaggle/input/waste-classification-data/DATASET/TEST"

In [3]:
# define fixed images size
IMG_SIZE = (384, 384)

In [4]:
# define image transformation function
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE,antialias=True),
    transforms.ToTensor(),
])

In [5]:
# create pytorch dataset using Image folder (each folder belongs to a class)
train_ds = ImageFolder(TRAIN_DIR,transform)
test_ds = ImageFolder(TEST_DIR,transform)

In [6]:
# get list of classes present in our dataset
cs = train_ds.classes
cs

['O', 'R']

In [7]:
BATCH_SIZE = 32

In [8]:
#cleanup memory before heavy training
import gc
gc.collect()

0

In [9]:
# create pytorch dataloader for batch training
train_loader =  DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader =  DataLoader(test_ds, batch_size=BATCH_SIZE) # val_loader doesn't requires shuffling

In [10]:
# download pretrained model(Resnet50) from torchhub for finetuning to garbage image data
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights="IMAGENET1K_V2")
model = model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 220MB/s]


In [11]:
# need to change classifier head, so get number of input features for it
num_ftrs = model.fc.in_features
num_ftrs

2048

In [12]:
# update classifier head to adjust to according to number of classes in our dataset
model.fc = nn.Linear(num_ftrs,len(cs))

In [13]:
def train(model,EPOCHS,train_loader,val_loader, lr = 0.001):

    #define training device (gpu/cpu) and total number of batches
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_n = len(train_loader)

    # define loss function, optimizer and move model to cuda if available
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model = model.to(device)

    # lists to record histories
    train_acc = []
    train_loss = []
    val_loss = [99]
    val_acc = [0]

    for epoch in range(EPOCHS):
        model.train(True)
        pbar = tqdm(train_loader)

        batch_loss = []
        batch_acc = []

        for i,data in enumerate(pbar):
            inputs,labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Make predictions for this batch
            outputs = model(inputs)
            acc = (labels == outputs.argmax(dim=-1)).float().mean().item()
            # Compute the loss and its gradients
            loss = criterion(outputs, labels)
            loss.backward()

            batch_loss.append(loss.item())
            batch_acc.append(acc)
            # Adjust learning weights
            optimizer.step()

            pbar.set_description(f"Epoch: {epoch + 1}/{EPOCHS}; Train Loss: {round(np.mean(batch_loss),3)}; Train Acc: {round(np.mean(batch_acc),3)};\
            Val Loss: {round(np.mean(val_loss),3)}; Val Acc: {round(np.mean(val_acc),3)}")

          # Set the model to evaluation mode, disabling dropout and using population
          # statistics for batch normalization.
        model.eval()
        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            val_batch_loss = []
            val_batch_acc = []
            for i, vdata in enumerate(val_loader):
                vinputs, vlabels = vdata
                vinputs = vinputs.to(device)
                vlabels = vlabels.to(device)
                voutputs = model(vinputs)
                test_acc = (vlabels == voutputs.argmax(dim=-1)).float().mean().item()
                vloss = criterion(voutputs, vlabels).item()

                val_batch_loss.append(vloss)
                val_batch_acc.append(test_acc)

        val_loss.append(np.mean(val_batch_loss))
        val_acc.append(np.mean(val_batch_acc))
        train_acc.append(np.mean(batch_acc))
        train_loss.append(np.mean(batch_loss))

    return train_acc,train_loss,val_acc,val_loss

In [14]:
EPOCHS = 10

In [15]:
train_acc,train_loss,val_acc,val_loss = train(model,EPOCHS,train_loader,val_loader,lr=1e-4)

Epoch: 1/10; Train Loss: 0.16; Train Acc: 0.941;            Val Loss: 99.0; Val Acc: 0.0: 100%|██████████| 706/706 [08:08<00:00,  1.44it/s]
Epoch: 2/10; Train Loss: 0.079; Train Acc: 0.971;            Val Loss: 49.591; Val Acc: 0.468: 100%|██████████| 706/706 [07:03<00:00,  1.67it/s]
Epoch: 3/10; Train Loss: 0.051; Train Acc: 0.983;            Val Loss: 33.146; Val Acc: 0.617: 100%|██████████| 706/706 [07:00<00:00,  1.68it/s]
Epoch: 4/10; Train Loss: 0.036; Train Acc: 0.986;            Val Loss: 24.924; Val Acc: 0.694: 100%|██████████| 706/706 [07:00<00:00,  1.68it/s]
Epoch: 5/10; Train Loss: 0.029; Train Acc: 0.99;            Val Loss: 20.012; Val Acc: 0.734: 100%|██████████| 706/706 [07:03<00:00,  1.67it/s]
Epoch: 6/10; Train Loss: 0.027; Train Acc: 0.992;            Val Loss: 16.732; Val Acc: 0.761: 100%|██████████| 706/706 [07:01<00:00,  1.67it/s]
Epoch: 7/10; Train Loss: 0.019; Train Acc: 0.994;            Val Loss: 14.384; Val Acc: 0.784: 100%|██████████| 706/706 [07:02<00:00,  1

In [16]:
torch.save(model.state_dict(), "garbage.pt")

In [17]:
import gc
gc.collect()

0

### :)) No time for EDA and VIZS